In [41]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
import JCfunc
import importlib
importlib.reload(JCfunc)

<module 'JCfunc' from 'C:\\Users\\GRIPS\\G-RIPS-2023-Mitsubishi-A\\Code\\AHP\\JCfunc.py'>

In [42]:
def adjust_angle(angle):
    # this function converts the angle so that if the angle is greater than pi it replaced with 2pi-angle
    if angle > 180:
        angle = 360 - angle
    return angle

In [43]:
gdf_utm = pd.read_pickle('gdf_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
curr_edge = pd.read_pickle('current_edge.pkl')

# for debugging purposes start at iteration point 15 
point_index = 12
# this is for debugging purposes, true 
# last_matched = gpd.GeoDataFrame(geometry=gpd.points_from_xy([819609.861], [5766244.800]), crs="EPSG:32631")

# current location 
curr_loc = gdf_utm.iloc[point_index].to_frame().T

# find longitude and latitude for last matched data
# last_matched['lon_lat'] = last_matched.to_crs({'init': 'epsg:4326'})

In [45]:
curr_loc

,id,time,phenomenons,geometry,GPS Speed,GPS HDOP,GPS Bearing,lon_lat,speed_mps
12,52f3836be4b0d8e8c27ed70a,2014-02-01 14:18:49+00:00,"{'GPS Bearing': {'value': 297.5, 'unit': 'deg'...",POINT (819609.7343499986 5766228.984348884),21.6,0.9,297.5,POINT (7.65214054 51.95470718),6.0


In [44]:
# extract node id
edge_node1 = nodes_utm.loc[curr_edge.index[0][0]]
edge_node2 = nodes_utm.loc[curr_edge.index[0][1]]

# select the end node 
if (curr_loc['GPS Bearing'].iloc[0] > 0 and curr_loc['GPS Bearing'].iloc[0] < 180):
    # if object movint to the right of map select the one with larger longitude
    if edge_node1.x > edge_node2.x:
        end_node = edge_node1
        start_node = edge_node2
    else:
        end_node = edge_node2
        start_node = edge_node1
else:
    # if object is moving to the left select node with smaller longitude
    if edge_node1.x < edge_node2.x:
        end_node = edge_node1
        start_node = edge_node2
    else:
        end_node = edge_node2
        start_node = edge_node1

# angle distance between the heading of the current point and the heading of the previous point
# it should lie between 0 to pi
delta_h = adjust_angle(abs(gdf_utm['GPS Bearing'].iloc[point_index] - gdf_utm['GPS Bearing'].iloc[point_index - 1]))

# distance traveled from the last position fix to the end nodes
d_1 = end_node['geometry'].distance(curr_loc['geometry']).iloc[0]

# distance traveled since the last position fix 
t = gdf_utm['time'].iloc[point_index] - gdf_utm['time'].iloc[point_index - 1]
d_2 = (gdf_utm['speed_mps'].iloc[point_index])* t.seconds

delta_d = d_1 - d_2

speed = curr_loc['speed_mps'].iloc[0]

MMLfunc.MML(delta_d, delta_h, speed)


False

In [40]:
def JC(trajectory_data, nodes_data, edges_data, current_position, previous_edge):
        # extract node id
    edge_node1 = nodes_data.loc[previous_edge.index[0][0]]
    edge_node2 = nodes_data.loc[previous_edge.index[0][1]]
    
    # select the end node 
    if (current_position['GPS Bearing'].iloc[0] > 0 and current_position['GPS Bearing'].iloc[0] < 180):
        # if object movint to the right of map select the one with larger longitude
        if edge_node1.x > edge_node2.x:
            end_node = edge_node1
            start_node = edge_node2
        else:
            end_node = edge_node2
            start_node = edge_node1
    else:
        # if object is moving to the left select node with smaller longitude
        if edge_node1.x < edge_node2.x:
            end_node = edge_node1
            start_node = edge_node2
        else:
            end_node = edge_node2
            start_node = edge_node1
    
    # angle distance between the heading of the current point and the heading of the previous point
    # it should lie between 0 to pi
    delta_h = adjust_angle(abs(trajectory_data['GPS Bearing'].iloc[iter] - trajectory_data['GPS Bearing'].iloc[iter - 1]))
    
    # distance traveled from the last position fix to the end nodes
    d_1 = end_node['geometry'].distance(current_position['geometry']).iloc[0]
    
    # distance traveled since the last position fix 
    t = trajectory_data['time'].iloc[iter] - trajectory_data['time'].iloc[iter - 1]
    d_2 = (trajectory_data['speed_mps'].iloc[iter])* t.seconds
    
    delta_d = d_1 - d_2
    
    speed = current_position['speed_mps'].iloc[0]

    iter = iter + 1
    
    return JCfunc.MML(delta_d, delta_h, speed)

In [38]:
iter = 12
JC(pd.read_pickle('gdf_utm.pkl'), pd.read_pickle('nodes_utm.pkl'), pd.read_pickle('edges_utm.pkl'), gdf_utm.iloc[iter].to_frame().T, pd.read_pickle('current_edge.pkl') )

False

In [39]:
iter 

13